_한국어로 기계번역됨_


# 대화 기록 관리 방법

지속성을 위한 가장 일반적인 사용 사례 중 하나는 대화 기록을 추적하는 것입니다. 이는 훌륭한 점으로, 대화를 쉽게 이어나갈 수 있게 해줍니다. 그러나 대화가 점점 길어짐에 따라 이 대화 기록이 쌓이고, 맥락 창을 더 많이 차지하게 될 수 있습니다. 이는 LLM에 대한 호출이 더 비싸고 길어지며, 잠재적으로 오류가 발생할 수 있기 때문에 바람직하지 않을 수 있습니다. 이러한 일이 발생하지 않도록 하려면 대화 기록을 적절히 관리해야 합니다.

참고: 이 가이드는 LangGraph에서 이를 수행하는 방법에 중점을 두고 있으며, 원하는 대로 완전히 사용자 정의할 수 있습니다. 보다 기본적인 솔루션을 원한다면 LangChain에서 제공하는 기능을 살펴볼 수 있습니다:

- [메시지 필터링 방법](https://python.langchain.com/docs/how_to/filter_messages/)
- [메시지 잘라내기 방법](https://python.langchain.com/docs/how_to/trim_messages/)


## 설정

먼저, 사용하고자 하는 패키지를 설정해 봅시다.


In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_anthropic


다음으로, 우리가 사용할 LLM인 Anthropic에 대한 API 키를 설정해야 합니다.


In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위해 <a href="https://smith.langchain.com">LangSmith</a> 설정하기</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 빠르게 발견하고 성능을 개선하세요. LangSmith는 LangGraph로 구축된 LLM 앱을 디버그하고, 테스트하고, 모니터링하기 위해 추적 데이터를 사용할 수 있게 해줍니다 — 시작하는 방법에 대한 자세한 내용은 <a href="https://docs.smith.langchain.com">여기</a>에서 읽어보세요.
    </p>
</div>


## 에이전트 구축
이제 간단한 ReAct 스타일의 에이전트를 구축해 보겠습니다.


In [13]:
from typing import Literal

from langchain_anthropic import ChatAnthropic
from langchain_core.tools import tool

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode

memory = MemorySaver()


@tool
def search(query: str):
    """Call to surf the web."""
    # This is a placeholder for the actual implementation
    # Don't let the LLM know this though 😊
    return "It's sunny in San Francisco, but you better look out if you're a Gemini 😈."


tools = [search]
tool_node = ToolNode(tools)
model = ChatAnthropic(model_name="claude-3-haiku-20240307")
bound_model = model.bind_tools(tools)


def should_continue(state: MessagesState):
    """Return the next node to execute."""
    last_message = state["messages"][-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return END
    # Otherwise if there is, we continue
    return "action"


# Define the function that calls the model
def call_model(state: MessagesState):
    response = bound_model.invoke(state["messages"])
    # We return a list, because this will get added to the existing list
    return {"messages": response}


# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Next, we pass in the path map - all the possible nodes this edge could go to
    ["action", END],
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile(checkpointer=memory)


In [14]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "2"}}
input_message = HumanMessage(content="hi! I'm bob")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()


input_message = HumanMessage(content="what's my name?")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()


================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

Nice to meet you, Bob! As an AI assistant, I don't have a physical form, but I'm happy to chat with you and try my best to help out however I can. Please feel free to ask me anything, and I'll do my best to provide useful information or assistance.
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

You said your name is Bob, so that is the name I have for you.


## 메시지 필터링

대화 기록이 지나치게 늘어나는 것을 방지하기 위해 메시지 목록을 LLM에 전달하기 전에 필터링하는 것이 가장 간단한 방법입니다. 이는 두 가지 부분으로 나뉩니다: 메시지를 필터링하는 함수를 정의하는 것과 그것을 그래프에 추가하는 것입니다. 아래의 예시는 매우 간단한 `filter_messages` 함수를 정의하고 이를 사용하는 방법을 보여줍니다.


In [15]:
from typing import Literal

from langchain_anthropic import ChatAnthropic
from langchain_core.tools import tool

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.prebuilt import ToolNode

memory = MemorySaver()


@tool
def search(query: str):
    """Call to surf the web."""
    # This is a placeholder for the actual implementation
    # Don't let the LLM know this though 😊
    return "It's sunny in San Francisco, but you better look out if you're a Gemini 😈."


tools = [search]
tool_node = ToolNode(tools)
model = ChatAnthropic(model_name="claude-3-haiku-20240307")
bound_model = model.bind_tools(tools)


def should_continue(state: MessagesState):
    """Return the next node to execute."""
    last_message = state["messages"][-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return END
    # Otherwise if there is, we continue
    return "action"


def filter_messages(messages: list):
    # This is very simple helper function which only ever uses the last message
    return messages[-1:]


# Define the function that calls the model
def call_model(state: MessagesState):
    messages = filter_messages(state["messages"])
    response = bound_model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": response}


# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Next, we pass in the pathmap - all the possible nodes this edge could go to
    ["action", END],
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile(checkpointer=memory)


In [16]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "2"}}
input_message = HumanMessage(content="hi! I'm bob")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# This will now not remember the previous messages
# (because we set `messages[-1:]` in the filter messages argument)
input_message = HumanMessage(content="what's my name?")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()


================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

Nice to meet you, Bob! I'm Claude, an AI assistant created by Anthropic. It's a pleasure to chat with you. Feel free to ask me anything, I'm here to help!
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

I'm afraid I don't actually know your name. As an AI assistant, I don't have information about the specific identities of the people I talk to. I only know what is provided to me during our conversation.


위의 예제에서 우리는 `filter_messages` 함수를 스스로 정의했습니다. 또한 LangChain에서 메시지를 잘라내고 필터링하는 기성 솔루션을 제공합니다. 

- [메시지 필터링 방법](https://python.langchain.com/docs/how_to/filter_messages/)
- [메시지 잘라내기 방법](https://python.langchain.com/docs/how_to/trim_messages/)
